# Final_Inactivity_time
- 전처리

In [17]:
#필요한 라이브러리 로드
import pandas as pd
import numpy as np
import datetime

In [18]:
#'union_201908.csv':'adid', 'datetime', 'partner_key',' event_name', 'amount'의 변수만 이용하였습니다.
df = pd.read_csv('union_1908.csv')
df.head()

,adid,datetime,partner_key,event_name,amount
0,0000a0276a0870de648c427edb012cba,2019-08-09 22:07:30.754,41loF2NK40GBUzUBDC1Zrw,NaN,NaN
1,0000a0276a0870de648c427edb012cba,2019-08-17 22:09:41.193,41loF2NK40GBUzUBDC1Zrw,NaN,NaN
2,0000ae28e29e570fd4cd3d1229b89ec2,2019-08-26 05:48:17.865,0BChBVxZSUOhg3yJgn7o0A,NaN,NaN
3,0000be453b034a699e09ba7ec1046fbf,2019-08-31 00:18:02.977,0BChBVxZSUOhg3yJgn7o0A,NaN,NaN
4,0001b3b01a0e9adfdfab4279212c4f2c,2019-08-13 14:09:15.755,T1m2A6iwPUemMh5gQZ3FRA,NaN,NaN


## Preprocessing 1
[partner_key]
- lookback_window팀과 partner_key를 통일하여, 해당 partner_key만 index로 지정하였습니다.
- 이후에는 한 session 내의 index들을 붙여서 path로 만들 것입니다. -> preprocessing 2로!

[timediff]
- datetime 간 간격을 계산하여 timediff를 만들고, 이것의 3분위수를 'inacticity'로 지정할 것입니다.
- timediff가 inactivity보다 작으면 한 session으로 보고, 크다면 다른 session으로 구분할 것입니다. -> preprocessing 2로!

#partner_key -> index

In [19]:
#'partner_key'의 NaN은 null로 채우기
df['partner_key'].fillna('null', inplace = True)
#'amount의 NaN은 0으로 채우기
df['amount'].fillna(0, inplace = True)

In [20]:
#partner_key는 lookback_window팀과 함께 통일함
partner_dic = {'T1m2A6iwPUemMh5gQZ3FRA':'a', 'NGHO984OM0SciKycDGPmYA':'b', 'RK10BMLBB02TTT0G0GWMrg':'c',
               '0BChBVxZSUOhg3yJgn7o0A':'d', '41loF2NK40GBUzUBDC1Zrw':'e', 'o72f0c4oF0WMsPGvuIt5vQ':'f', 'null':'direct'}
#해당 지정한 partner_key만 index로 지정
for key, value in partner_dic.items():
    df.loc[df['partner_key'] == key, 'index'] = value

#datetime -> timediff

In [25]:
#datetime 간 간격을 계산하는 새로운 'timediff' 컬럼 만들기
df['datetime'] = pd.to_datetime(df['datetime'])
df['timediff'] = df['datetime'].diff().dt.days * 24*3600 + (df['datetime'].diff().dt.seconds)

In [26]:
#adid가 바뀌는 지점의 timediff는 NaN으로 만들기
id_dic = {}
for key, value in df.adid.items():
    id_dic[value] = key+1
change = list(id_dic.values())[:-1]
df.loc[change, 'timediff'] = np.nan

In [27]:
#우리가 사용할 기준 시간은 'inactivity'로, timediff의 3분위수임. 출력하면:
inactivity = df.timediff.quantile(0.75)
print(inactivity)

272650.0


## Preprocessing 2 : micro_session
이제 session을 구분해주는 구분자인 'micro_session'을 정의합니다. micro_session을 이용하여, 이것의 값이 0이면 같은 session이므로 하나의 path로 묶고, 1이라면 다른 session이므로 다른 path로 구분합니다.

[조건]
- 시작: micro_session을 0으로 설정한다. (첫 번째 micro_session은 항상 1이다.)
- 조건 1.  adid가 바뀌면 1이다.
- 조건 2.  purchase 다음 행은 1이다.
- 조건 3.  timediff > inactivity이면 1이다.

In [8]:
#시작
df['micro_session'] = 0
df['new_micro_session'] = 0
df.loc[0, 'micro_session'] = 1 

#조건 1.
df.loc[change, 'micro_session'] = 1

#조건 2.
purchase_change = list(map(lambda x: int(1) if x == "abx:purchase" else np.nan, list(df['event_name'])))
purchase_change = [i+1 for i in range(len(purchase_change)) if purchase_change[i] == 1]
if purchase_change[-1] == len(df):
    purchase_change = purchase_change[:-1]
df.loc[purchase_change, 'micro_session'] = 1

#조건 3.
timediff_change = list(map(lambda x: int(1) if x >= inactivity else np.nan, list(df['timediff'])))
timediff_change = [i for i in range(len(timediff_change)) if timediff_change[i] == 1]
df.loc[timediff_change, 'micro_session'] = 1

#new micro session
minimum_time_change = list(map(lambda x: int(1) if x <= 5 else np.nan, list(df['timediff'])))
minimum_time_change = [i for i in range(len(minimum_time_change)) if minimum_time_change[i] == 1]
df.loc[minimum_time_change, 'new_micro_session'] = 1

In [15]:
df.head(50)

,adid,datetime,partner_key,event_name,amount,index,timediff,micro_session,new_micro_session
0,0000a0276a0870de648c427edb012cba,2019-08-09 22:07:30.754,41loF2NK40GBUzUBDC1Zrw,NaN,0.0,e,NaN,1,0
1,0000a0276a0870de648c427edb012cba,2019-08-17 22:09:41.193,41loF2NK40GBUzUBDC1Zrw,NaN,0.0,e,192.036111,1,0
2,0000ae28e29e570fd4cd3d1229b89ec2,2019-08-26 05:48:17.865,0BChBVxZSUOhg3yJgn7o0A,NaN,0.0,d,NaN,1,0
3,0000be453b034a699e09ba7ec1046fbf,2019-08-31 00:18:02.977,0BChBVxZSUOhg3yJgn7o0A,NaN,0.0,d,NaN,1,0
4,0001b3b01a0e9adfdfab4279212c4f2c,2019-08-13 14:09:15.755,T1m2A6iwPUemMh5gQZ3FRA,NaN,0.0,a,NaN,1,0
5,0001b3b01a0e9adfdfab4279212c4f2c,2019-08-18 09:32:24.286,T1m2A6iwPUemMh5gQZ3FRA,NaN,0.0,a,115.385556,1,0
6,0001b3b01a0e9adfdfab4279212c4f2c,2019-08-18 09:32:25.131,T1m2A6iwPUemMh5gQZ3FRA,NaN,0.0,a,0.000000,0,1
7,0001b3b01a0e9adfdfab4279212c4f2c,2019-08-18 09:32:25.343,T1m2A6iwPUemMh5gQZ3FRA,NaN,0.0,a,0.000000,0,1
8,0001b3b01a0e9adfdfab4279212c4f2c,2019-08-19 07:17:06.501,T1m2A6iwPUemMh5gQZ3FRA,NaN,0.0,a,21.744722,0,0
9,0001b3b01a0e9adfdfab4279212c4f2c,2019-08-20 04:11:55.896,T1m2A6iwPUemMh5gQZ3FRA,NaN,0.0,a,20.913611,0,0


## Preprocessing 3
- 완성되는 전처리 완료된 파일은 path별로 total_conversions, total_conversion_value, total_null 값을 계산하여 반환하는 데이터프레임입니다.

In [10]:
ms = list(df['micro_session'])
ns = list(df['new_micro_session'])
ix = list(df['index'])
mt = list(df['amount'])

path = []
amount = [] #purchase가 발생했을 때의 금액입니다.

#micro_session이 1이면 다른 session이고, 
#micro_session이 0이면 같은 session이므로 한 path로 묶어 반환합니다. amount의 합 또한 계산합니다.
for p, n, i, m in zip(ms, ns, ix, mt):
    if p == 1:
        path.append([i])
        amount.append([m])
    elif p == 0:
        if n == 0:
            path[-1].append(i)
        amount[-1].append(m)
        
path = ['>'.join(p) for p in path]
amount = [sum(m) for m in amount]
conversion = [1 if p.endswith('direct') else 0 for p in path]
null = [1 if c == 0 else 0 for c in conversion]

fin_df = pd.DataFrame({'path': path, 'total_conversions': conversion, 'total_conversion_value': amount, 'total_null': null})

In [11]:
pp = [path[:-7] if '>direct' in path else path for path in list(fin_df['path'])]
fin_df['path'] = pp

fin_df2 = fin_df[fin_df.path !='direct']  ##여기서 direct로만 이루어진 행은 뺐다! 갠찮나?

In [16]:
fin_df2.head(50)

,path,total_conversions,total_conversion_value,total_null
0,e,0,0.0,1
1,e,0,0.0,1
2,d,0,0.0,1
3,d,0,0.0,1
4,a,0,0.0,1
5,a>a>a>a,0,0.0,1
7,d,0,0.0,1
8,d,1,97200.0,0
12,d,0,0.0,1
13,a,0,0.0,1


In [13]:
fin_df2.to_csv('test_1.csv', index = False)